In [118]:
import xarray as xr
import pandas as pd
from pathlib import Path

data_dir = "./GFED5/monthly/" 
start_date = "2018-04"
end_date = "2020-12" 
aggregation = "mean"

# Convert dates to datetime for comparison
start_date = pd.to_datetime(start_date, format="%Y-%m")
end_date = pd.to_datetime(end_date, format="%Y-%m")

# List all NetCDF files in the directory

all_files = sorted(Path(data_dir).glob("*.nc"))

# Filter files by date range

filtered_files = [
     str(file) for file in all_files
     if start_date.year <= int(file.stem[-4:]) <= end_date.year
     ]

if not filtered_files:
     raise ValueError("No files match the specified date range.")

print(f"Found {len(filtered_files)} files in the date range.")

Found 3 files in the date range.


In [119]:
import xarray as xr
import pandas as pd
from pathlib import Path

data_dir = "./GFED5/monthly/" 
start_date = pd.to_datetime("2018-04", format="%Y-%m")
end_date = pd.to_datetime("2020-12", format="%Y-%m")
aggregation = "mean"

year_start = start_date.year
year_end = end_date.year


all_files = sorted(Path(data_dir).glob("*.nc"))

# Filter files by date range

filtered_files = [
     str(file) for file in all_files
     if year_start <= int(file.stem[-4:]) <= year_end
     ]

filtered_files

['GFED5/monthly/GFED5_Beta_monthly_2018.nc',
 'GFED5/monthly/GFED5_Beta_monthly_2019.nc',
 'GFED5/monthly/GFED5_Beta_monthly_2020.nc']

In [120]:
# Open the dataset and select only one data variable
var = "C"
ds = xr.open_mfdataset(filtered_files)[var]


# Aggregate the data by the specified method
ds = ds.mean(dim="time")
# ds2 = ds2.sel(
#     lat=slice(37, -35),
#     lon=slice(-17, 51)
# )

ds

<xarray.DataArray 'C' (lat: 720, lon: 1440)> Size: 4MB
dask.array<mean_agg-aggregate, shape=(720, 1440), dtype=float32, chunksize=(720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 3kB 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
  * lon      (lon) float32 6kB -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9

In [156]:
import h3
import xarray as xr
import pandas as pd
from collections import defaultdict

lat = ds['lat'].values
lon = ds['lon'].values
data = ds.values

# Define H3 resolution
resolution = 4

# Prepare lists to store results
hex_ids = []
values = []

# Iterate over the data array to aggregate into hexagons
for lat_idx in range(len(lat)):
    for lon_idx in range(len(lon)):
        value = data[lat_idx, lon_idx]
        if value > 0:  # Only process positive values
            hex_id = h3.latlng_to_cell(lat[lat_idx], lon[lon_idx], resolution)
            hex_ids.append(hex_id)
            values.append(value)

# Create a dictionary for aggregation
hex_dict = defaultdict(float)
for hex_id, value in zip(hex_ids, values):
    hex_dict[hex_id] += value

df = pd.DataFrame(hex_dict.items(), columns=["hex_id", "value"])
df['value'] = df['value'].astype(float).round(2)

print(df)

                hex_id  value
0      8406369ffffffff   0.19
1      8405aedffffffff   3.05
2      8405ae7ffffffff   2.84
3      8405aadffffffff   0.68
4      840b6bbffffffff   0.08
...                ...    ...
50978  84df691ffffffff   0.01
50979  84df6b1ffffffff   0.20
50980  84df6b7ffffffff   0.02
50981  84df6b5ffffffff   0.01
50982  84df4c9ffffffff   0.10

[50983 rows x 2 columns]


In [ ]:
# plot using pydeck h3_hexagon_layer

import pydeck as pdk

# Define the layer

layer = pdk.Layer(
     "H3HexagonLayer",
     data=df,
     pickable=True,
     stroked=False,
     filled=True,
     get_hexagon="hex_id",
     get_fill_color="[255, 255 - value, 0]",
     )


# Set the viewport location
view_state = pdk.ViewState(
     latitude=0,
     longitude=0,
     zoom=2,
     bearing=0,
     pitch=0,
     )


r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "Emission(grams): {value}"})
r



{
  "initialViewState": {
    "bearing": 0,
    "latitude": 0,
    "longitude": 0,
    "pitch": 0,
    "zoom": 2
  },
  "layers": [
    {
      "@@type": "H3HexagonLayer",
      "data": [
        {
          "hex_id": "820637fffffffff",
          "value": 0.19
        },
        {
          "hex_id": "8205a7fffffffff",
          "value": 116.92
        },
        {
          "hex_id": "8205affffffffff",
          "value": 5.56
        },
        {
          "hex_id": "820b6ffffffffff",
          "value": 28.47
        },
        {
          "hex_id": "820587fffffffff",
          "value": 41.99
        },
        {
          "hex_id": "820417fffffffff",
          "value": 137.6
        },
        {
          "hex_id": "82041ffffffffff",
          "value": 416.48
        },
        {
          "hex_id": "820b4ffffffffff",
          "value": 5.97
        },
        {
          "hex_id": "8204affffffffff",
          "value": 1266.32
        },
        {
          "hex_id": "820b47fffffffff",
          "value": 16.97
        },
        {
          "hex_id": "82058ffffffffff",
          "value": 182.19
        },
        {
          "hex_id": "8205b7fffffffff",
          "value": 612.07
        },
        {
          "hex_id": "8204a7fffffffff",
          "value": 4205.35
        },
        {
          "hex_id": "820d0ffffffffff",
          "value": 1.15
        },
        {
          "hex_id": "820a5ffffffffff",
          "value": 24.87
        },
        {
          "hex_id": "8204f7fffffffff",
          "value": 1894.15
        },
        {
          "hex_id": "82059ffffffffff",
          "value": 1728.41
        },
        {
          "hex_id": "820127fffffffff",
          "value": 1.21
        },
        {
          "hex_id": "820d77fffffffff",
          "value": 3.47
        },
        {
          "hex_id": "821327fffffffff",
          "value": 0.97
        },
        {
          "hex_id": "820a57fffffffff",
          "value": 1.09
        },
        {
          "hex_id": "8204c7fffffffff",
          "value": 1099.42
        },
        {
          "hex_id": "8204effffffffff",
          "value": 0.69
        },
        {
          "hex_id": "8208cffffffffff",
          "value": 20.75
        },
        {
          "hex_id": "820137fffffffff",
          "value": 6.84
        },
        {
          "hex_id": "820a77fffffffff",
          "value": 14.15
        },
        {
          "hex_id": "821167fffffffff",
          "value": 12.02
        },
        {
          "hex_id": "820597fffffffff",
          "value": 1524.45
        },
        {
          "hex_id": "8204cffffffffff",
          "value": 77.86
        },
        {
          "hex_id": "820db7fffffffff",
          "value": 0.56
        },
        {
          "hex_id": "82132ffffffffff",
          "value": 0.03
        },
        {
          "hex_id": "82095ffffffffff",
          "value": 0.02
        },
        {
          "hex_id": "82106ffffffffff",
          "value": 50.25
        },
        {
          "hex_id": "820b57fffffffff",
          "value": 439.8
        },
        {
          "hex_id": "820b5ffffffffff",
          "value": 9.85
        },
        {
          "hex_id": "820aeffffffffff",
          "value": 390.57
        },
        {
          "hex_id": "820daffffffffff",
          "value": 15.69
        },
        {
          "hex_id": "820d1ffffffffff",
          "value": 8.53
        },
        {
          "hex_id": "820d47fffffffff",
          "value": 1022.87
        },
        {
          "hex_id": "8208effffffffff",
          "value": 1.58
        },
        {
          "hex_id": "820487fffffffff",
          "value": 1796.85
        },
        {
          "hex_id": "82048ffffffffff",
          "value": 3800.21
        },
        {
          "hex_id": "820d67fffffffff",
          "value": 3.6
        },
        {
          "hex_id": "821147fffffffff",
          "value": 10.2
        },
        {
          "hex_id": "820a2ffffffffff",
          "value": 

In [17]:
import rioxarray as rio
import numpy as np
import leafmap


# ds2 = ds2.where(ds2 != 0, np.nan)



# ds2 = ds2.rio.write_crs("EPSG:4326")
# ds2 = ds2.rio.set_spatial_dims(x_dim="lon", y_dim="lat")

# ds2.rio.to_raster("data.tif", dtype="float32")
# # do hexagon binning from the raster data




# m = leafmap.Map(zoom=2)
# m.add_raster(
#     "data.tif", 
#     layer_name="Aggregated Data", 
#     colormap="ocean_r",
#     opacity=0.9
# )

# m